In [40]:
import xarray as xr
import numpy as np
from numpy import pi, sin, cos, arccos, clip, deg2rad
import numpy.ma as ma
from datetime import datetime
import dask
import time

@dask.delayed
def load(dsmapper):
    
    """
    Load data from zarr-store
    """
    ds = xr.open_zarr(dsmapper, consolidated=True, chunks='auto')
    data0 = ds.Anomalies.load().values[:,0]
    ii = ~xr.apply_ufunc(np.isnan, data0)
    data, lat, lon, time = data0[ii], ds.latitude.load().values[ii], ds.longitude.load().values[ii], ds.time.load().values[ii]
    
    ### Calculation using data-, lat-, lon-, and time-arrays
    ###
    ### Returns one array
    return None
    #return data, lat, lon, time

### Connect to a dask-cluster and set cluster-options

In [3]:
import json
import gcsfs

with open('pangeo-181919-e7bc5bdaf4d5.json') as f:
    token = json.load(f)
gcs = gcsfs.GCSFileSystem(token=token)

from dask_gateway import GatewayCluster, Gateway
from distributed import Client

g = Gateway()
#g.list_clusters()

In [4]:
options = g.cluster_options()
options.worker_cores = 2; options.worker_memory = 4
# Create a cluster with those options
cluster = g.new_cluster(options)

In [5]:
g.list_clusters()

[ClusterReport<name=prod.fbded122d1854cd1a0243d9421dc55e1, status=RUNNING>]

In [6]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.fbded122d1854cd1a0243d9421dc55e1/status,


In [7]:
cluster.scale(1)

### Loading of data-, lat-, lon-, and time-arrays from cloud-storage

Loading dataset only:

In [8]:
plevel = 4
dspath='pangeo-argo-eke/global/mean_and_anomalies/with_pressure_coordinate/readable_ws_and_NHarm/mean_and_anomalies_global_ws500_plevel'+str(plevel)+'.zarr'
dsmapper = gcs.get_mapper(dspath)

#### Alternatives:

In [26]:
%%time 
results = dask.compute( load(dsmapper) )

CPU times: user 9.37 ms, sys: 792 µs, total: 10.2 ms
Wall time: 1.58 s


In [39]:
%%time 
results = dask.compute( load(dsmapper) )

CPU times: user 9.96 ms, sys: 0 ns, total: 9.96 ms
Wall time: 1.54 s


In [45]:
%%time 
results = dask.compute( load(dsmapper) )

CPU times: user 8.83 ms, sys: 58 µs, total: 8.89 ms
Wall time: 1.54 s


Dataset looks like this:

In [46]:
xr.open_zarr(dsmapper, consolidated=True)

<xarray.Dataset>
Dimensions:    (i: 2105319, pressure: 1)
Coordinates:
    latitude   (i) float64 dask.array<chunksize=(2105319,), meta=np.ndarray>
    longitude  (i) float64 dask.array<chunksize=(2105319,), meta=np.ndarray>
  * pressure   (pressure) int64 50
    time       (i) datetime64[ns] dask.array<chunksize=(2105319,), meta=np.ndarray>
Dimensions without coordinates: i
Data variables:
    Anomalies  (i, pressure) float64 dask.array<chunksize=(2105319, 1), meta=np.ndarray>
    Mean       (i, pressure) float64 dask.array<chunksize=(2105319, 1), meta=np.ndarray>
Attributes:
    creation_date:        2023-07-07 11:37:05.311220
    description:          Estimated mean dynamic height on profile-coordinate...
    number_of_harmonics:  2
    pressureindex:        4
    window_size:          500000.0

#### Scaling down and closing cluster

In [47]:
cluster.scale(0)

In [ ]:
cluster.close()

In [ ]:
cluster.shutdown()